In [1]:
# Dependencies
import matplotlib.pyplot as plt

import os
import numpy as np
import tensorflow as tf

import keras
from keras.preprocessing import image
from keras.applications.xception import (
    Xception, preprocess_input, decode_predictions)
from keras.applications.vgg19 import (
    VGG19, preprocess_input, decode_predictions)

import cv2                 
from random import shuffle 
from tqdm import tqdm      

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf














# VGG19 -------------------------------------------------------------------------------------------------------------------------------------

# Load the VGG19 model
Vmodel = VGG19(
    include_top=True,
    weights='imagenet')

# Default Image Size for VGG19
VGG_image_size = (224, 224)

# Refactor above steps into reusable function
def Vpredict(image_path):
    """Use VGG19 to label image"""
    img = image.load_img(image_path, target_size=VGG_image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = Vmodel.predict(x)
    plt.imshow(img)
    print('Predicted:', decode_predictions(predictions, top=3)[0])






# /VGG19 ------------------------------------------------------------------------------------------------------------------------------------








# Xception ----------------------------------------------------------------------------------------------------------------------------------

# Load the Xception model
Xmodel = Xception(
    include_top=True,
    weights='imagenet')

# Default Image Size for Xception
Xception_image_size = (299, 299)


# Reusable function to call on given photo
def Xpredict(image_path):
    """Use Xception to label image"""
    img = image.load_img(image_path, target_size=Xception_image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = Xmodel.predict(x)
    plt.imshow(img)
    print('Predicted:', decode_predictions(predictions, top=3)[0])

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN_DIR = '../Image_Data/train'
TEST_DIR = '../Image_Data/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'model'
# just so we remember which saved model is which, sizes must match

def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

train_data = np.load('train_data.npy')


tf.reset_default_graph()
def train():
    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 128, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.8)

    convnet = fully_connected(convnet, 2, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

    model = tflearn.DNN(convnet, tensorboard_dir='log')



    if os.path.exists('C:/Users/H/Desktop/KaggleDogsvsCats/{}.meta'.format(MODEL_NAME)):
        model.load(MODEL_NAME)
        print('model loaded!')

    train = train_data[:-500]
    test = train_data[-500:]

    X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
    Y = [i[1] for i in train]

    test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
    test_y = [i[1] for i in test]

    model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
        snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    
    model.save('model.tflearn')

In [3]:
def run():
    test_data = np.load('test_data.npy')
    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
    model = tflearn.DNN(convnet, tensorboard_dir='log')
    model.load('model.tflearn')
    for num,data in enumerate(test_data[:1]):
        # cat: [1,0]
        # dog: [0,1]
        
        img_num = data[1]
        img_data = data[0]
        
        
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        
        if np.argmax(model_out) == 1: Vpredict('../Image_Data/test/1.jpg')
        else: Xpredict('../Image_Data/test/1.jpg')

In [4]:
train()

Training Step: 3829  | total loss: 0.34576 | time: 34.405s
| Adam | epoch: 010 | loss: 0.34576 - acc: 0.8400 -- iter: 24448/24500
Training Step: 3830  | total loss: 0.34374 | time: 35.485s
| Adam | epoch: 010 | loss: 0.34374 - acc: 0.8420 | val_loss: 0.49564 - val_acc: 0.7740 -- iter: 24500/24500
--
INFO:tensorflow:/Users/rob/Projects/Local_Workspace/Sarahs_Angels/Models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
run()

INFO:tensorflow:Restoring parameters from /Users/rob/Projects/Local_Workspace/Sarahs_Angels/Models/model.tflearn


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Accuracy/Mean/moving_avg_1 not found in checkpoint
	 [[{{node save_5/RestoreV2}} = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_BOOL, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_5/Const_0_0, save_5/RestoreV2/tensor_names, save_5/RestoreV2/shape_and_slices)]]

Caused by op 'save_5/RestoreV2', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python